In [1]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tf_slim as slim

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
# Root directory of the project
ROOT_DIR = "C:/python_class/Building detection/"

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "Logs")

In [4]:
class CustomConfig(Config):
    """Configuration for training on the building image  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"

    
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + Building


    # Number of training steps per epoch
    STEPS_PER_EPOCH = 10

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

In [5]:
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir):
      
      
        # Add classes. We have only one class to add.
        self.add_class("object", 1, "building")
        

     
        # Train or validation dataset?
        
        dataset_dir = os.path.join(dataset_dir)
        
        # We mostly care about the x and y coordinates of each region
        
        
        annotations1 = json.load(open(os.path.join(dataset_dir, 'train_json (4).json')))
        
        
        # print(annotations1)
        annotations = list(annotations1.values())  



        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        count=0
        for a in annotations:
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            count = count +1
            polygons = [r['shape_attributes']for r in a['regions']] 
            objects = [s['region_attributes']['building'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"building": 1}
            # key = tuple(name_dict)
            num_ids = [name_dict[a] for a in objects]
            
            # num_ids = [int(n['Event']) for n in objects]
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            print("numids",num_ids)
            image_path = os.path.join('C:/python_class/Building detection/New folder (2)', a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]
            print("Count : ",count)

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self._class_, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self._class_, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self._class_, self).image_reference(image_id)

In [6]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom("C:/python_class/Building detection/")
    
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom("C:/python_class/Building detection/")
    dataset_val.prepare()

   
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=100,
                layers='heads')

In [7]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

train(model)

objects: ['building', 'building', 'building', 'building']
numids [1, 1, 1, 1]
Count :  1
objects: ['building', 'building', 'building']
numids [1, 1, 1]
Count :  2
objects: ['building', 'building', 'building', 'building', 'building', 'building', 'building', 'building', 'building', 'building', 'building', 'building', 'building']
numids [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Count :  3
objects: ['building', 'building', 'building', 'building', 'building']
numids [1, 1, 1, 1, 1]
Count :  4
objects: ['building', 'building', 'building', 'building']
numids [1, 1, 1, 1]
Count :  5
objects: ['building', 'building', 'building', 'building', 'building', 'building', 'building', 'building']
numids [1, 1, 1, 1, 1, 1, 1, 1]
Count :  6
objects: ['building', 'building', 'building']
numids [1, 1, 1]
Count :  7
objects: ['building', 'building', 'building', 'building', 'building', 'building']
numids [1, 1, 1, 1, 1, 1]
Count :  8
objects: ['building', 'building', 'building']
numids [1, 1, 1]
Count :  9
obje

c:\Users\formy\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\framework\indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


10/10 [==============================] - ETA: 0s - batch: 4.5000 - size: 1.0000 - loss: 2.6450 - rpn_class_loss: 0.1310 - rpn_bbox_loss: 0.5390 - mrcnn_class_loss: 0.4540 - mrcnn_bbox_loss: 0.9325 - mrcnn_mask_loss: 0.5885 WARNING:tensorflow:From c:\Users\formy\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\training_v1.py:2048: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
10/10 [==============================] - 1062s 106s/step - batch: 4.5000 - size: 1.0000 - loss: 2.6450 - rpn_class_loss: 0.1310 - rpn_bbox_loss: 0.5390 - mrcnn_class_loss: 0.4540 - mrcnn_bbox_loss: 0.9325 - mrcnn_mask_loss: 0.5885 - val_loss: 2.0469 - val_rpn_class_loss: 0.0738 - val_rpn_bbox_loss: 0.4802 - val_mrcnn_class_loss: 0.3102 - val_mrcnn_bbox_loss: 0.7170 - val_mrcnn_mask_loss: